# Advent of Haystack: Day 8

_Make a copy of this Colab to start!_

In this challenge, we will create an Agent for Santa's backoffice: a powerful assistant capable of answering questions about the gift inventory, tracking items taken for delivery, and purchasing new ones.

We will use several Haystack components, focusing primarily on the new experimental **🛠️ Tool support** (which will soon be merged into the main repository).
It's not completely documented yet, but you can find the most important information in this [GitHub discussion](https://github.com/deepset-ai/haystack-experimental/discussions/98).

**Some Useful Components**

- [DuckduckgoApiWebSearch](https://haystack.deepset.ai/integrations/duckduckgo-api-websearch) or another [WebSearch](https://docs.haystack.deepset.ai/docs/websearch) component
- [PromptBuilder](https://docs.haystack.deepset.ai/docs/promptbuilder)
- [OpenAIGenerator](https://docs.haystack.deepset.ai/docs/openaigenerator) or any other `Generator`
- 🧪 [OpenAIChatGenerator](https://github.com/deepset-ai/haystack-experimental/blob/813157dd75cc95275c51d90bc6cfb7382d88ccc2/haystack_experimental/components/generators/chat/openai.py#L88)
- 🧪 [ToolInvoker](https://docs.haystack.deepset.ai/reference/experimental-tools-api#toolinvoker)


## 1) Installation


In [1]:
! uv pip install -U openai haystack-ai duckduckgo-api-haystack

Resolved 41 packages in 1.10s                                        
Audited 41 packages in 0.32ms


## 2) Enter your API key

Enter your OpenAI API key to use the `OpenAIGenerator` and `OpenAIChatGenerator`. Alternatively, you can explore and use other [Generators](https://docs.haystack.deepset.ai/docs/generators) with different models and providers.


In [2]:
from getpass import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass("Enter OpenAI API key:")

### (Optional) Setup the `LoggingTracer`

We recently introduced [Real-Time Pipeline Logging](https://docs.haystack.deepset.ai/docs/logging#real-time-pipeline-logging), that allows to easily inspect the data that's flowing through your pipelines. Particularly helpful during experimentation with complex pipelines.


In [3]:
import logging
from haystack import tracing
from haystack.tracing.logging_tracer import LoggingTracer

logging.basicConfig(
    format="%(levelname)s - %(name)s -  %(message)s", level=logging.WARNING
)
logging.getLogger("haystack").setLevel(logging.DEBUG)

tracing.tracer.is_content_tracing_enabled = (
    True  # to enable tracing/logging content (inputs/outputs)
)
tracing.enable_tracing(
    LoggingTracer(
        tags_color_strings={
            "haystack.component.input": "\x1b[1;31m",
            "haystack.component.name": "\x1b[1;34m",
        }
    )
)

## 3) Populate the inventory

In this section, we use a simple Haystack [`InMemoryDocumentStore`](https://docs.haystack.deepset.ai/docs/inmemorydocumentstore) as our inventory.
The gift/items will be `Documents`.


In [4]:
from haystack.document_stores.in_memory import InMemoryDocumentStore
from haystack import Document

document_store = InMemoryDocumentStore()

In [5]:
documents = [
    Document(
        content="LEGO Star Wars Set",
        meta={"units": 3456, "origin": "Amazon", "description": "Amazon"},
    ),
    Document(
        content="Wooden Sailboat",
        meta={"units": 124, "origin": "handmade", "description": "Handmade"},
    ),
    Document(
        content="Nintendo Switch",
        meta={"units": 2189, "origin": "Amazon", "description": "Amazon"},
    ),
    Document(
        content="Hand-Knitted Teddy Bear",
        meta={"units": 233, "origin": "handmade", "description": "Handmade"},
    ),
    Document(
        content="Barbie Dreamhouse",
        meta={"units": 1673, "origin": "Amazon", "description": "Amazon"},
    ),
    Document(
        content="Carved Wooden Puzzle",
        meta={"units": 179, "origin": "handmade", "description": "Handmade"},
    ),
    Document(
        content="Remote Control Drone",
        meta={"units": 1542, "origin": "Amazon", "description": "Amazon"},
    ),
    Document(
        content="Painted Rocking Horse",
        meta={"units": 93, "origin": "handmade", "description": "Handmade"},
    ),
    Document(
        content="Science Experiment Kit",
        meta={"units": 2077, "origin": "Amazon", "description": "Amazon"},
    ),
    Document(
        content="Miniature Dollhouse",
        meta={"units": 110, "origin": "handmade", "description": "Handmade"},
    ),
    Document(
        content="Nerf Blaster",
        meta={"units": 2731, "origin": "Amazon", "description": "Amazon"},
    ),
    Document(
        content="Interactive Robot Pet",
        meta={"units": 1394, "origin": "Amazon", "description": "Amazon"},
    ),
]

In [6]:
document_store.write_documents(documents)

12

## 4) Tools

Our Santa's backoffice Agent need several Tools to work, each one with its specific action:

- look up an item in inventory
- add item to inventory
- take item from inventory
- inventory summary
- get price of a new item
- buy a new item

We are going to create them, with your help.
For an introduction to Tools, check out [Cookbook: Define & Run Tools](https://haystack.deepset.ai/cookbook/tools_support).


### Lookup tool

This is used to find if an item is present in the inventory.
We will use a [`InMemoryBM25Retriever`](https://docs.haystack.deepset.ai/docs/inmemorybm25retriever) to allow also not exact matches.


In [ ]:
from haystack_experimental.dataclasses import Tool
from typing import Annotated, Literal

from haystack.components.retrievers.in_memory import InMemoryBM25Retriever

retriever = InMemoryBM25Retriever(document_store=document_store, top_k=3)

DEBUG - haystack.core.component.component -  Registering <class 'haystack.components.retrievers.filter_retriever.FilterRetriever'> as a component
DEBUG - haystack.core.component.component -  Registered Component <class 'haystack.components.retrievers.filter_retriever.FilterRetriever'>
DEBUG - haystack.core.component.component -  Registering <class 'haystack.components.retrievers.in_memory.bm25_retriever.InMemoryBM25Retriever'> as a component
DEBUG - haystack.core.component.component -  Registered Component <class 'haystack.components.retrievers.in_memory.bm25_retriever.InMemoryBM25Retriever'>
DEBUG - haystack.core.component.component -  Registering <class 'haystack.components.retrievers.in_memory.embedding_retriever.InMemoryEmbeddingRetriever'> as a component
DEBUG - haystack.core.component.component -  Registered Component <class 'haystack.components.retrievers.in_memory.embedding_retriever.InMemoryEmbeddingRetriever'>
DEBUG - haystack.core.component.component -  Registering <class 'h

After creating the retriever, we define a function that converts the search results to text, ready to be crunched by Language Models.

As you can notice, we annotate the arguments in the function signature and provide a detailed docstring to make the conversion to a Tool seamless.
To learn this trick, take a look at the [Newsletter Sending Agent notebook](https://haystack.deepset.ai/cookbook/newsletter-agent#extras-converting-tools).


In [8]:
def lookup_item_in_inventory(item_name: Annotated[str, "The item name to search"]):
    """
    Look up an item in the inventory.
    """
    result = retriever.run(query=item_name)
    text = ""
    for doc in result["documents"]:
        text += f"found item: {doc.content}; units: {doc.meta['units']}; matching score: {doc.score}\n"
    return text

In [9]:
print(lookup_item_in_inventory(item_name="lego"))

found item: LEGO Star Wars Set; units: 3456; matching score: 2.3976626592085233
found item: Wooden Sailboat; units: 124; matching score: 1.3496776558458576
found item: Nintendo Switch; units: 2189; matching score: 1.3496776558458576



In [ ]:
lookup_item_in_inventory_tool = Tool.from_function(lookup_item_in_inventory)

In [11]:
print(lookup_item_in_inventory_tool.invoke(item_name="lego"))

found item: LEGO Star Wars Set; units: 3456; matching score: 2.3976626592085233
found item: Wooden Sailboat; units: 124; matching score: 1.3496776558458576
found item: Nintendo Switch; units: 2189; matching score: 1.3496776558458576



### Add item tool

Next, a tool to add an item to the inventory


In [12]:
from haystack.document_stores.types import DuplicatePolicy


def add_item_to_inventory(
    item_name: Annotated[str, "The item name to add to inventory"],
    origin: Annotated[Literal["handmade", "Amazon"], "The origin of the item"],
    units: Annotated[int, "The number of units to add to inventory"] = 1,
):
    """
    Add an item to the inventory.
    """
    found = document_store.filter_documents(
        filters={"field": "content", "operator": "==", "value": item_name}
    )
    id_ = None
    if found:
        units += found[0].meta["units"]
        id_ = found[0].id

    doc = Document(id=id_, content=item_name, meta={"units": units, "origin": origin})
    return document_store.write_documents([doc], policy=DuplicatePolicy.OVERWRITE)

In [13]:
add_item_to_inventory_tool = Tool.from_function(add_item_to_inventory)

### Inventory Summary tool

Now it's your turn.

Let's start with a basic `inventory_summary` function and its `inventory_summary_tool`.

This tool is expected to retrieve all items and return a textual summary/list as `"name: <NAME>; units: <UNITS>; origin: <ORIGIN>"` for each item.


In [21]:
def inventory_summary():
    """
    Get a summary of the inventory.
    """
    summary = ""
    for doc in document_store.filter_documents(filters={}):
        summary += f"name: {doc.content}; units: {doc.meta['units']}; origin: {doc.meta['origin']}\n"

    return summary

In [22]:
print(inventory_summary())

name: LEGO Star Wars Set; units: 3456; origin: Amazon
name: Wooden Sailboat; units: 124; origin: handmade
name: Nintendo Switch; units: 2189; origin: Amazon
name: Hand-Knitted Teddy Bear; units: 233; origin: handmade
name: Barbie Dreamhouse; units: 1673; origin: Amazon
name: Carved Wooden Puzzle; units: 179; origin: handmade
name: Remote Control Drone; units: 1542; origin: Amazon
name: Painted Rocking Horse; units: 93; origin: handmade
name: Science Experiment Kit; units: 2077; origin: Amazon
name: Miniature Dollhouse; units: 110; origin: handmade
name: Nerf Blaster; units: 2731; origin: Amazon
name: Interactive Robot Pet; units: 1394; origin: Amazon



In [23]:
inventory_summary_tool = Tool.from_function(inventory_summary)

### Take from Inventory tool

A more complex tool for you to build!

This should take as input the `item_name` and the `units`.

- it should try to fetch the item
- if not present, return a message saying `"item {item_name} not found in inventory"`
- if present and units > units in inventory, return a message saying `"item {item_name} has only {units_in_inventory} units, cannot take {units}"`
- otherwise, remove the specified `units` from the inventory and return an explanatory message saying `"item {item_name} has been updated in inventory"`


In [24]:
def take_from_inventory(item_name: Annotated[str, "The item name to take from inventory"],
                          units: Annotated[int, "The number of units to take from inventory"]=1,
                        ):
    """
    Take an item to the inventory.
    """
    found=document_store.filter_documents(filters={"field": "content", "operator": "==", "value": item_name})
    id_ = None
    
    if not found:
        return f"Item {item_name} not found in inventory"
    
    if units > found[0].meta["units"]:
        return f"Item {item_name} has only {found[0].meta['units']} units, cannot take {units} units"
      
    units = found[0].meta["units"] - units
    id_ = found[0].id
    
    doc = Document(id=id_, content=item_name, meta={"units": units, "origin": found[0].meta["origin"]})
    document_store.write_documents([doc], policy=DuplicatePolicy.OVERWRITE)  
    return f"Item {item_name} has been updated in inventory"

In [27]:
print(take_from_inventory(item_name="LEGO Star Wars Set", units=1))

Item LEGO Star Wars Set has been updated in inventory


In [28]:
take_from_inventory_tool = Tool.from_function(take_from_inventory)

### Get Price tool

This tool tries to find the Amazon price of the item in the web.

In this case, the tool wraps a Web RAG Pipeline.
The tool is given but you need to define the pipeline with [DuckduckgoApiWebSearch](https://haystack.deepset.ai/integrations/duckduckgo-api-websearch), [PromptBuilder](https://docs.haystack.deepset.ai/docs/promptbuilder) and [OpenAIGenerator](https://docs.haystack.deepset.ai/docs/openaigenerator).

**HINT:** If you quickly hit the rate limit, you can change the `backend` of `DuckduckgoApiWebSearch` and use "html" (or "lite").


In [62]:
from haystack import Pipeline
from haystack.components.builders.prompt_builder import PromptBuilder
from haystack.components.generators import OpenAIGenerator
from duckduckgo_api_haystack import DuckduckgoApiWebSearch


template = """Given the information below, answer the query. Only use the provided context to generate the answer and output the used document links
            Context:
            {% for document in documents %}
                {{ document.content }}
                URL: {{ document.meta.url }}
            {% endfor %}

            Question: {{ query }}
            Answer: """

web_search = DuckduckgoApiWebSearch(backend="lite")
prompt_builder = PromptBuilder(template = template)
generator = OpenAIGenerator(model="gpt-4o-mini")


In [63]:
### DEFINE THE WEB RAG PIPELINE HERE ###
get_price_pipe = Pipeline()
get_price_pipe.add_component(name="search" , instance=web_search)
get_price_pipe.add_component(name="prompt_builder", instance=prompt_builder)
get_price_pipe.add_component(name="llm", instance=generator)

DEBUG - haystack.core.pipeline.base -  Adding component 'search' (<duckduckgo_api_haystack.duckduckgoapi.DuckduckgoApiWebSearch object at 0x7f1c1d9d59d0>

Inputs:
  - query: str
Outputs:
  - documents: List[Document]
  - links: List[str])
DEBUG - haystack.core.pipeline.base -  Adding component 'prompt_builder' (<haystack.components.builders.prompt_builder.PromptBuilder object at 0x7f1c1d9d5af0>

Inputs:
  - documents: Any
  - query: Any
  - template: Optional[str]
  - template_variables: Optional[Dict[str, Any]]
Outputs:
  - prompt: str)
DEBUG - haystack.core.pipeline.base -  Adding component 'llm' (<haystack.components.generators.openai.OpenAIGenerator object at 0x7f1c24780620>

Inputs:
  - prompt: str
  - system_prompt: Optional[str]
  - streaming_callback: Optional[Callable[]]
  - generation_kwargs: Optional[Dict[str, Any]]
Outputs:
  - replies: List[str]
  - meta: List[Dict[str, Any]])


In [64]:
get_price_pipe.connect("search.documents", "prompt_builder.documents")
get_price_pipe.connect("prompt_builder.prompt", "llm.prompt")

DEBUG - haystack.core.pipeline.base -  Connecting 'search.documents' to 'prompt_builder.documents'
DEBUG - haystack.core.pipeline.base -  Connecting 'prompt_builder.prompt' to 'llm.prompt'


🚅 Components
  - search: DuckduckgoApiWebSearch
  - prompt_builder: PromptBuilder
  - llm: OpenAIGenerator
🛤️ Connections
  - search.documents -> prompt_builder.documents (List[Document])
  - prompt_builder.prompt -> llm.prompt (str)

In [69]:
def get_price(item_name: Annotated[str, "The item name to search"]):
    """
    Search the web to get the price of an item on Amazon
    """

    search_query = f"price of {item_name} on Amazon"
    question = f"What is the price of {item_name} on Amazon? ONLY Respond with minimal item name and minimum price."

    data = {"search": {"query": search_query}, "prompt_builder": {"query": question}}

    return get_price_pipe.run(data=data)["llm"]["replies"][0]

In [70]:
get_price("barbie dollhouse")

INFO - haystack.core.pipeline.pipeline -  Running component search
DEBUG - haystack.tracing.logging_tracer -  Operation: haystack.component.run
DEBUG - haystack.tracing.logging_tracer -  haystack.component.name=search
DEBUG - haystack.tracing.logging_tracer -  haystack.component.type=DuckduckgoApiWebSearch
DEBUG - haystack.tracing.logging_tracer -  haystack.component.input_types={'query': 'str'}
DEBUG - haystack.tracing.logging_tracer -  haystack.component.input_spec={'query': {'type': 'str', 'senders': []}}
DEBUG - haystack.tracing.logging_tracer -  haystack.component.output_spec={'documents': {'type': 'typing.List[haystack.dataclasses.document.Document]', 'receivers': ['prompt_builder']}, 'links': {'type': 'typing.List[str]', 'receivers': []}}
DEBUG - haystack.tracing.logging_tracer -  haystack.component.input={'query': 'price of barbie dollhouse on Amazon'}
DEBUG - haystack.tracing.logging_tracer -  haystack.component.visits=1
DEBUG - haystack.tracing.logging_tracer -  haystack.comp

'Barbie DreamHouse, Doll House Playset - $114.99\n\nUsed Document Links:\n1. URL: \n2. URL: \n3. URL: '

In [71]:
get_price_tool = Tool.from_function(get_price)

### Buy from Amazon tool

This tool is ready to use.

It asks the user for confirmation and then simulates a purchase on Amazon. It also adds items to the inventory.


In [72]:
def buy_from_amazon(
    item_name: Annotated[str, "The item name to search"],
    price: Annotated[float, "The price of the item to buy"],
    units: Annotated[int, "The number of units to buy"] = 1,
):
    """
    Buy an item from Amazon and place it in the inventory.
    """

    total_price = units * price
    confirm = input(
        f"You are about to buy {units} units of {item_name} from Amazon for a total of ${total_price}. Are you sure you want to continue? (y/n)"
    )
    if confirm == "y":
        # simulate actually buying from Amazon
        add_item_to_inventory(item_name, units=units, origin="Amazon")
        return "transaction completed and item added to inventory"

    return "transaction cancelled"

In [73]:
buy_from_amazon_tool = Tool.from_function(buy_from_amazon)

In [74]:
buy_from_amazon(item_name="Playstation 5", price=500.00, units=5)

'transaction completed and item added to inventory'

## 5) Main loop

This part controls the flow of the application.
It is quite simple and you can use to see the Agent in action and check that everything is working properly. For the Agent, you will use the experimental versions of `OpenAIChatGenerator` and `ChatMessage`.

**Note:** You can use experimental versions of `OllamaChatGenerator`, `HuggingFaceAPIChatGenerator` and `AnthropicChatGenerator` instead of `OpenAIChatGenerator`. See all experimental `Generators` [here](https://github.com/deepset-ai/haystack-experimental/tree/main/haystack_experimental/components/generators)

To understand what's happening, it is important to be familiar with the experimental `ChatMessage` dataclass (see this [Cookbook: Define & Run Tools](https://haystack.deepset.ai/cookbook/tools_support)).

---

If every missing part has been implemented correctly, the Agent should be able to answer questions and perform actions like the following:

```
What's in the inventory?
I take 1300 Barbie Dreamhouse and 50 Wooden Sailboat
Buy 50 Harry Potter and the Philosopher's Stone books from Amazon
Buy 50 Doom 3 videogames; then I take 40 of them
Price of Bose noise removing headphones
I want to add 27 Wooden trains handmade by elves
```


In [75]:
from haystack_experimental.components.generators.chat import OpenAIChatGenerator
from haystack_experimental.components.tools.tool_invoker import ToolInvoker
from haystack_experimental.dataclasses import ChatMessage

tools = [
    lookup_item_in_inventory_tool,
    add_item_to_inventory_tool,
    inventory_summary_tool,
    take_from_inventory_tool,
    get_price_tool,
    buy_from_amazon_tool,
]

chat_generator = OpenAIChatGenerator(tools=tools)

tool_invoker = ToolInvoker(tools=tools)
messages = [
    ChatMessage.from_system(
        """You manage Santa Claus backoffice. Always talk with a XMAS tone and references. You are expected to talk with Santas elves.
            Prepare a tool call if needed, otherwise use your knowledge to respond to the user.
            If the invocation of a tool requires the result of another tool, prepare only one call at a time.

            Each time you receive the result of a tool call, ask yourself: "Am I done with the task?".
            If not and you need to invoke another tool, prepare the next tool call.
            If you are done, respond with just the final result."""
    )
]

while True:
    user_input = input("\n\nwaiting for input (type 'exit' or 'quit' to stop)\n🧝: ")
    if user_input.lower() == "exit" or user_input.lower() == "quit":
        break
    messages.append(ChatMessage.from_user(user_input))

    while True:
        print("⌛ iterating...")

        replies = chat_generator.run(messages=messages)["replies"]
        messages.extend(replies)

        # Check for tool calls and handle them
        if not replies[0].tool_calls:
            break
        tool_calls = replies[0].tool_calls

        tool_messages = tool_invoker.run(messages=replies)["tool_messages"]
        messages.extend(tool_messages)

    # Print the final AI response after all tool calls are resolved
    print(f"🤖: {messages[-1].text}")

DEBUG - haystack.core.component.component -  Registering <class 'haystack.components.preprocessors.document_cleaner.DocumentCleaner'> as a component
DEBUG - haystack.core.component.component -  Registered Component <class 'haystack.components.preprocessors.document_cleaner.DocumentCleaner'>
DEBUG - haystack.core.component.component -  Registering <class 'haystack.components.preprocessors.document_splitter.DocumentSplitter'> as a component
DEBUG - haystack.core.component.component -  Registered Component <class 'haystack.components.preprocessors.document_splitter.DocumentSplitter'>
DEBUG - haystack.core.component.component -  Registering <class 'haystack.components.preprocessors.nltk_document_splitter.NLTKDocumentSplitter'> as a component
DEBUG - haystack.core.component.component -  Registered Component <class 'haystack.components.preprocessors.nltk_document_splitter.NLTKDocumentSplitter'>
DEBUG - haystack.core.component.component -  Registering <class 'haystack.components.preprocessors

⌛ iterating...
⌛ iterating...
🤖: Ho ho ho! 🎅 Here's a merry list of what's jingling in our inventory:

- **Wooden Sailboat:** 124 units (handmade)
- **Nintendo Switch:** 2189 units (Amazon)
- **Hand-Knitted Teddy Bear:** 233 units (handmade)
- **Barbie Dreamhouse:** 1673 units (Amazon)
- **Carved Wooden Puzzle:** 179 units (handmade)
- **Remote Control Drone:** 1542 units (Amazon)
- **Painted Rocking Horse:** 93 units (handmade)
- **Science Experiment Kit:** 2077 units (Amazon)
- **Miniature Dollhouse:** 110 units (handmade)
- **Nerf Blaster:** 2731 units (Amazon)
- **Interactive Robot Pet:** 1394 units (Amazon)
- **LEGO Star Wars Set:** 3455 units (Amazon)
- **Playstation 5:** 5 units (Amazon)

Let me know if you need anything else, my jolly friend! 🎄✨
⌛ iterating...
⌛ iterating...
🤖: Ho ho ho! 🎁 The inventory has been updated with your joyful request! We’ve successfully taken:

- 1300 **Barbie Dreamhouse** 
- 50 **Wooden Sailboat** 

If there's anything else you need to spread the Ch

INFO - haystack.core.pipeline.pipeline -  Running component search
DEBUG - haystack.tracing.logging_tracer -  Operation: haystack.component.run
DEBUG - haystack.tracing.logging_tracer -  haystack.component.name=search
DEBUG - haystack.tracing.logging_tracer -  haystack.component.type=DuckduckgoApiWebSearch
DEBUG - haystack.tracing.logging_tracer -  haystack.component.input_types={'query': 'str'}
DEBUG - haystack.tracing.logging_tracer -  haystack.component.input_spec={'query': {'type': 'str', 'senders': []}}
DEBUG - haystack.tracing.logging_tracer -  haystack.component.output_spec={'documents': {'type': 'typing.List[haystack.dataclasses.document.Document]', 'receivers': ['prompt_builder']}, 'links': {'type': 'typing.List[str]', 'receivers': []}}
DEBUG - haystack.tracing.logging_tracer -  haystack.component.input={'query': "price of Harry Potter and the Philosopher's Stone on Amazon"}
DEBUG - haystack.tracing.logging_tracer -  haystack.component.visits=1
DEBUG - haystack.tracing.logging

⌛ iterating...
⌛ iterating...
🤖: Hooray! 🎉 The magical journey begins! We've successfully bought **50 copies of "Harry Potter and the Philosopher's Stone"** from Amazon, and they are now added to our festive inventory! 

If you need anything else to keep the holiday spirit bright, just say the word! 📚✨🎄
⌛ iterating...
⌛ iterating...
🤖: Jingle all the way! 🎅 Here's the cheerful inventory update after our recent festive activities:

- **Nintendo Switch:** 2189 units (Amazon)
- **Hand-Knitted Teddy Bear:** 233 units (handmade)
- **Carved Wooden Puzzle:** 179 units (handmade)
- **Remote Control Drone:** 1542 units (Amazon)
- **Painted Rocking Horse:** 93 units (handmade)
- **Science Experiment Kit:** 2077 units (Amazon)
- **Miniature Dollhouse:** 110 units (handmade)
- **Nerf Blaster:** 2731 units (Amazon)
- **Interactive Robot Pet:** 1394 units (Amazon)
- **LEGO Star Wars Set:** 3455 units (Amazon)
- **Playstation 5:** 5 units (Amazon)
- **Barbie Dreamhouse:** 373 units (Amazon)
- **Woode

INFO - haystack.core.pipeline.pipeline -  Running component search
DEBUG - haystack.tracing.logging_tracer -  Operation: haystack.component.run
DEBUG - haystack.tracing.logging_tracer -  haystack.component.name=search
DEBUG - haystack.tracing.logging_tracer -  haystack.component.type=DuckduckgoApiWebSearch
DEBUG - haystack.tracing.logging_tracer -  haystack.component.input_types={'query': 'str'}
DEBUG - haystack.tracing.logging_tracer -  haystack.component.input_spec={'query': {'type': 'str', 'senders': []}}
DEBUG - haystack.tracing.logging_tracer -  haystack.component.output_spec={'documents': {'type': 'typing.List[haystack.dataclasses.document.Document]', 'receivers': ['prompt_builder']}, 'links': {'type': 'typing.List[str]', 'receivers': []}}
DEBUG - haystack.tracing.logging_tracer -  haystack.component.input={'query': 'price of Doom 3 video game on Amazon'}
DEBUG - haystack.tracing.logging_tracer -  haystack.component.visits=1
DEBUG - haystack.tracing.logging_tracer -  haystack.com

⌛ iterating...


INFO - haystack.core.pipeline.pipeline -  Running component search
DEBUG - haystack.tracing.logging_tracer -  Operation: haystack.component.run
DEBUG - haystack.tracing.logging_tracer -  haystack.component.name=search
DEBUG - haystack.tracing.logging_tracer -  haystack.component.type=DuckduckgoApiWebSearch
DEBUG - haystack.tracing.logging_tracer -  haystack.component.input_types={'query': 'str'}
DEBUG - haystack.tracing.logging_tracer -  haystack.component.input_spec={'query': {'type': 'str', 'senders': []}}
DEBUG - haystack.tracing.logging_tracer -  haystack.component.output_spec={'documents': {'type': 'typing.List[haystack.dataclasses.document.Document]', 'receivers': ['prompt_builder']}, 'links': {'type': 'typing.List[str]', 'receivers': []}}
DEBUG - haystack.tracing.logging_tracer -  haystack.component.input={'query': 'price of Doom 3 on Amazon'}
DEBUG - haystack.tracing.logging_tracer -  haystack.component.visits=1
DEBUG - haystack.tracing.logging_tracer -  haystack.component.outp

⌛ iterating...
⌛ iterating...
⌛ iterating...
🤖: Hooray! 🎉 We've successfully bought **50 copies of Doom 3** for $28.74 each, and they are now in our inventory! Plus, I've taken **40 copies** just for you!

If you have more requests to keep the holiday cheer going, feel free to ask! 🎄✨
⌛ iterating...


INFO - haystack.core.pipeline.pipeline -  Running component search
DEBUG - haystack.tracing.logging_tracer -  Operation: haystack.component.run
DEBUG - haystack.tracing.logging_tracer -  haystack.component.name=search
DEBUG - haystack.tracing.logging_tracer -  haystack.component.type=DuckduckgoApiWebSearch
DEBUG - haystack.tracing.logging_tracer -  haystack.component.input_types={'query': 'str'}
DEBUG - haystack.tracing.logging_tracer -  haystack.component.input_spec={'query': {'type': 'str', 'senders': []}}
DEBUG - haystack.tracing.logging_tracer -  haystack.component.output_spec={'documents': {'type': 'typing.List[haystack.dataclasses.document.Document]', 'receivers': ['prompt_builder']}, 'links': {'type': 'typing.List[str]', 'receivers': []}}
DEBUG - haystack.tracing.logging_tracer -  haystack.component.input={'query': 'price of Bose noise cancelling headphones on Amazon'}
DEBUG - haystack.tracing.logging_tracer -  haystack.component.visits=1
DEBUG - haystack.tracing.logging_tracer 

⌛ iterating...
🤖: Jingle bells, jingle bells! 🎧 The price for the **Bose Noise Cancelling Headphones** is **$199.00**! 

If you’d like to add some of these delightful headphones to our inventory or need anything else to jingle your holiday spirit, just let me know! 🎄✨
⌛ iterating...
⌛ iterating...
🤖: Hooray for handmade wonders! 🎉 I've added **27 Wooden Trains**, crafted by our magical elves, to the inventory!

If there's anything else you'd like to spread the Christmas cheer or any other requests, just let me know! 🎄✨
